# A `qit` implementation of the CHIPPR algorithm

_Eric Charles & Alex Malz_

This notebook demonstrates a real use case of `qit`, to reproduce the results of [CHIPPR](https://arxiv.org/abs/2007.12178).

In [ ]:
# imports
import numpy as np
import qp
import qit
from scipy.optimize import minimize
%matplotlib inline

## True redshift distribution $n^{\dagger}(z)$

First, we make a true distribution.  
For this simple example, it is just a Gaussian

`true_dist` = $n^{\dagger}(z) = p(z | \phi^{\dagger})$

In [ ]:
# true distribution of redshifts
Z_TRUE_MIN, Z_TRUE_MAX = 0., 2.
LOC_TRUE = 0.60
SCALE_TRUE = 0.30

true_dist = qp.Ensemble(qp.stats.norm, data=dict(loc=LOC_TRUE, scale=SCALE_TRUE))
ax_true = true_dist.plot(xlim=(Z_TRUE_MIN, Z_TRUE_MAX), label=r"unnorm")

## Implicit prior $n^{*}(z)$

Now we make the implicit prior.  
In our case it is similiar to the true redshift distribution, but slightly different.

implicit_prior = $n^{*}(z) = p(z|\phi^{*})$

In [ ]:
LOC_PRIOR = 0.65
SCALE_PRIOR = 0.35
implicit_prior = qp.Ensemble(qp.stats.norm, data=dict(loc=LOC_PRIOR, scale=SCALE_PRIOR))
ax_prior = implicit_prior.plot(xlim=(Z_TRUE_MIN, Z_TRUE_MAX), label=r"unnorm")

## Model of physical likelihoods $p(d | z)$

Now we try a simplistic model for the behavior of the space of photometry and redshift $p(z, d)$.
A likelihood $p(d | z)$ in this space would correspond to a distribution over possible data $d$ for a true value $z$.
$d$ would be multidimensional in real life, one dimension per photometric filter, but we're assuming there's a unique projection of the photometry into a univariate $d$.

`likelihood` is $\mathcal{L}(d | i)$           # Where i is a bin index

`like_estim` is $\mathcal{L}(d | z_{\rm true})$   # This does the lookup to map z_true -> i

**Note: `likelihood` and `like_estim` might be better named `like_binned`/`like_discrete` and `like_continuous`/`like_true` respectively**

In [ ]:
# This represents the true model of photometry and redshift, we define 50 bins (in true redshift)
# and in each bin the likelihood p(z_obs) is a Gaussian centered on the bin center

N_EST_BINS = 50

z_bins = np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_EST_BINS+1)
z_centers = qp.utils.edge_to_center(z_bins)
z_widths = 0.2 * np.ones(N_EST_BINS)
likelihood = qp.Ensemble(qp.stats.norm, data=dict(loc=np.expand_dims(z_centers, -1), scale=np.expand_dims(z_widths, -1)))
like_estim = qit.Estimator([z_bins], likelihood)

In [ ]:
# These are the photometric data points at which we will evaluate posterior PDFs
N_OBS_BINS = 300
Z_OBS_MIN, Z_OBS_MAX = -0.5, 2.5
grid_edge = np.linspace(Z_OBS_MIN, Z_OBS_MAX, N_OBS_BINS+1)
grid_cent = qp.utils.edge_to_center(grid_edge)
p_grid = likelihood.pdf(grid_cent)

plot_kwds = dict(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                 ylim=(Z_OBS_MIN, Z_OBS_MAX), 
                 xlabel=r'$z_{\rm true}$',
                 ylabel=r'$d$')
pl_like = qit.plotting.plot_2d_like(p_grid.T, **plot_kwds)

## Per-galaxy posterior distributions

Ok, now we are going to extract the posterior distributions 

true posteriors of the physical model = `post_grid` = $p(z|d)$

estimated posteriors for an estimator with implicit prior $\phi^{*}$ = `est_grid` = $p(z|d,\phi^{*})$ 

true posteriors of the galaxy sample = `true_grid` = $p(z|d,\phi^{\dagger})$. 

In our case these correspond to the posteriors assuming a flat prior, assuming the true distribution as the prior and assuming the implicit prior. **I am confused. What does the preceding sentence mean?**

In [ ]:
like_estim = qit.Estimator([z_bins], likelihood)
flat_post = like_estim.flat_posterior(grid_edge)

In [ ]:
# Let's flip around the likelihood
z_grid = np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, 101)

post_grid = flat_post.get_posterior_grid(z_grid)
est_grid = flat_post.get_posterior_grid(z_grid, implicit_prior)
true_grid = flat_post.get_posterior_grid(z_grid, true_dist)

pl_post = qit.plotting.plot_2d_like(post_grid, **plot_kwds)
pl_est = qit.plotting.plot_2d_like(est_grid, **plot_kwds)
pl_true = qit.plotting.plot_2d_like(true_grid, **plot_kwds)

**Note: plots above should have axis labels swapped accordingly.**

## Sample galaxy redshifts from the true redshift distribution $n^{\dagger}(z)$

Here we make a histogram of 10000 true redshifts sampled from $p(z|\phi^{\dagger})$

In [ ]:
# Now let's sample points in true z
N_SAMPLES = 10000
N_HIST_BINS = 50
z_true_sample = np.squeeze(true_dist.rvs(size=N_SAMPLES))
fig_sample, ax_sample = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                     xlabel=r"$z_{\rm true}$",
                                                     ylabel="Counts / %0.2f" % ((Z_TRUE_MAX-Z_TRUE_MIN)/N_HIST_BINS))
hist = ax_sample.hist(z_true_sample, bins=np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_HIST_BINS+1))

## Create a sample of photometric "data" of the physical model

We do this by sampling a $d$ value from the correct bin for each sampled value in $z_{\rm true}$.

In [ ]:
# Now we create a sample of points in measured z.
N_OBS_HIST_BINS = 75

sampler = like_estim.get_sampler(z_true_sample)
mask = (z_true_sample > 0) * (z_true_sample <= 2.0)
z_meas_sample = np.squeeze(sampler.rvs(1))

fig_hmeas, ax_hmeas = qp.plotting.make_figure_axes(xlim=(Z_OBS_MIN, Z_OBS_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel="Counts / %0.2f" % ((Z_OBS_MAX-Z_OBS_MIN)/N_OBS_HIST_BINS))

hist = ax_hmeas.hist(z_meas_sample, bins=np.linspace(Z_OBS_MIN, Z_OBS_MAX, N_OBS_HIST_BINS+1))

**Note: x axis above should be $d$.**

In [ ]:
# Overplot the scatter plot on the 2-d likelihood plot
pl_true = qit.plotting.plot_2d_like(p_grid.T, **plot_kwds)
ax_like2 = pl_true[1]
sc = ax_like2.scatter(z_true_sample[mask], z_meas_sample, s=1, color='gray')

## Profile plot of likelihoods

The previous plot is a bit messy, lets plot the mean and std in slices of x.  (This is a "profile" plot in particle physics jargon.)

In [ ]:
N_PROF_BINS = 20
pl_true2 = qit.plotting.plot_2d_like(p_grid.T, **plot_kwds)
ax_prof = pl_true2[1]
x_prof = np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_PROF_BINS+1)
x_prof_cent = qp.utils.edge_to_center(x_prof)
prof_vals, prof_errs = qp.utils.profile(z_true_sample[mask], z_meas_sample, x_prof)
sc = ax_prof.errorbar(x_prof_cent, prof_vals, yerr=prof_errs)

## Posteriors for the measured values

Now we make "Posterior" objects

`post_p` = $p(z|d_{j})$ 

`est_p` = $p(z | d_{j}, \phi^{*})$ 

`true_p` = $p(z | d_{j}, \phi^{\dagger})$ 

for the samples we simulated.   Note that this is different than we made the grids above.   Now the index $j$ runs over the samples, i.e., for 10000 sampled galaxies we have 10000 entries in each of `post_p`, `est_p` and `true_p`, corresponding to the posterior PDFs for each of the sampled data values $d_j$.

In [ ]:
# Now we get the redshift posteriors for all the measured data
z_meas_bin = np.searchsorted(grid_edge, z_meas_sample)-1
z_meas_mask = (z_meas_bin >= 0) * (z_meas_bin < grid_cent.size)
z_meas_bin = z_meas_bin[z_meas_mask]

post_p = like_estim.make_posterior(z_grid, z_true_sample)
est_p = like_estim.make_posterior(z_grid, z_true_sample, implicit_prior)
true_p = like_estim.make_posterior(z_grid, z_true_sample, true_dist)

def make_dict(post, z_grid):
    ens = post._ensemble
    vals = ens.pdf(z_grid)
    return dict(post=post, ens=ens, vals=vals, stack=vals.mean(axis=0))

post_dict = make_dict(post_p, z_grid)
est_dict = make_dict(est_p, z_grid)
true_dict = make_dict(true_p, z_grid)

In [ ]:
# Let's plot a particular event from the sample, 
# so this one galaxy's posterior under the physical model of redshift and photometry, 
# under an estimation model with implicit prior $\phi^{*}$,
# and under a (very lucky) estimation model with implicit prior $\phi^{\dagger}$
which_sample = np.argmax(z_meas_sample[0:100])
fig_x, ax_x = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                           xlabel=r"$z_{\rm true}$",
                                           ylabel=r"$p(z)$")
ax_x.plot(z_grid, post_dict['vals'][which_sample], label='posterior under flat implicit prior')
ax_x.plot(z_grid, est_dict['vals'][which_sample], label='posterior under estimator\'s implicit prior')
ax_x.plot(z_grid, true_dict['vals'][which_sample], label='posterior under true redshift distribution as implict prior')
ax_x.plot(z_grid, np.squeeze(implicit_prior.pdf(z_grid)), label=r'implicit prior = estimator\'s $n^{*}(z)$')
ax_x.plot(z_grid, np.squeeze(true_dist.pdf(z_grid)), label=r'implicit prior = true $n^{\dagger}(z)$')
leg = fig_x.legend()

## Check of effect of binning the samples

This compares a histogram made from the original $d$ values to a histogram made by taking the closest grid point

In [ ]:
fig_check, ax_check = qp.plotting.make_figure_axes(xlim=(Z_OBS_MIN, Z_OBS_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel="Counts / %0.02f" % ((Z_OBS_MAX-Z_OBS_MIN)/N_OBS_HIST_BINS))

ax_check.hist(z_meas_sample, bins=np.linspace(Z_OBS_MIN, Z_OBS_MAX, N_OBS_HIST_BINS+1), label='sample', histtype='step')
z_meas_binned = grid_cent[z_meas_bin]
ax_check.hist(z_meas_binned, bins=np.linspace(Z_OBS_MIN, Z_OBS_MAX, N_OBS_HIST_BINS+1), label='check', histtype='step')
leg = fig_check.legend()

## Compare the true redshift distribution to the naive "stacking"

**Note: check interpretation that either implicit prior tightens distribution relative to physical model, and that the shift in mean is due to differences between using estimated vs. true n(z) as implicit prior.**

In [ ]:
N_FIT_BINS = 4
hist_bins = np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_FIT_BINS+1)
fig_stack, ax_stack = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel=r"$p(z)$")
ax_stack.hist(z_true_sample[mask], bins=hist_bins, density=True, label=r'$z_{\rm true}$', histtype='step')
ax_stack.plot(z_grid, np.squeeze(true_dist.pdf(z_grid)), label=r'$p(z) = n^{\dagger}(z)$')
ax_stack.plot(z_grid, post_dict['stack'], label=r'$\sum_{j} p(z | d_{j})$')
ax_stack.plot(z_grid, est_dict['stack'], label=r'$\sum_{j} p(z | d_{j}, \phi^{\dagger})$')
ax_stack.plot(z_grid, true_dict['stack'], label=r'$\sum_{j} p(z | d_{j}, \phi^{*})$')
leg = fig_stack.legend()


## Plot some per-galaxy redshift posterior distributions

**Note: check interpretation that implicit prior tightens distribution relative to physical model**

In [ ]:
fig_1, ax_1 = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel=r"$p(z | d)$")
fig_2, ax_2 = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel=r"$p(z | d, \phi^{*})$")

post_vals = post_dict['vals']
est_vals = est_dict['vals']
for i in range(10):
    ax_1.plot(z_grid, post_vals[i])
    ax_2.plot(z_grid, est_vals[i])

## Test to make sure we can update the parameters of a PDF for fitting

This is just a software test to make sure that setting the values of a model parameter changes the model.

In [ ]:
model_params = np.ones((1, N_FIT_BINS))
model = qp.Ensemble(qp.stats.hist, data=dict(bins=hist_bins, pdfs=model_params))
fig_model, ax_model = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                   xlabel=r"$z_{\rm true}$",
                                                   ylabel=r"$p(z)$")

ax_model.plot(z_grid, np.squeeze(model.pdf(z_grid)), label='orig')

new_params = np.ones(N_FIT_BINS)
new_params[1] = 1.4

model.update_objdata(dict(pdfs=np.expand_dims(new_params, 0)))
ax_model.plot(z_grid, np.squeeze(model.pdf(z_grid)), label='new')
leg = fig_model.legend()

# Test the likelihood function by evaluating it for a flat distribution and for the true redshift distribution

Here "model" is our model of the implicit prior

model = $p(z|\alpha)$ , where $\alpha$ are the hyper-parameters for our prior

And "llike" is the likelihood associated to that model, computed by summing over the sample

llike = $\sum_i \ln \int P(z_{\rm true}|d_i) \frac{p(z|\alpha)}{p(z|\phi)}  dz_{\rm true}$

In [ ]:
N_EVAL_PTS = 201
eval_grid = np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_EVAL_PTS)
model_params = np.log(np.ones(N_FIT_BINS))
hist_cents = qp.utils.edge_to_center(hist_bins)
true_vals = np.histogram(z_true_sample, bins=np.linspace(Z_TRUE_MIN, Z_TRUE_MAX, N_FIT_BINS+1))[0]
llike = qit.Likelihood(model, est_dict['post'], eval_grid)
v_flat = llike(model_params)
v_true = llike(np.log(true_vals))
print(v_flat, v_true)

# Fit for the hyper-parameters

In [ ]:
result = minimize(llike, model_params)
print(result)

In [ ]:
# Check the current value of the objective function
llike(result['x'])

In [ ]:
# Extract the parameters and convert back to counts (The Jacobian happens to be identical to the fitted values)
fitted_vals = np.exp(result['x'])
fitted_errs = np.sqrt(np.array([result['hess_inv'][i,i] for i in range(4)]))
norm_factor = 2 / fitted_vals.sum()
normed_fit = norm_factor * fitted_vals
#jac = fitted_vals
# Convert to PDF, for plotting
#normed_errs = norm_factor * jac * fitted_errs
model.update_objdata(dict(pdfs=np.expand_dims(normed_fit, 0)))
model_vals = np.squeeze(model.pdf(z_grid))

In [ ]:
fig_result, ax_result = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                     xlabel=r"$z_{\rm true}$",
                                                     ylabel=r"$p(z)$")
ax_result.hist(z_true_sample[mask], bins=hist_bins, density=True, label=r'$z_{\rm true}$', histtype='step')
ax_result.plot(z_grid, np.squeeze(true_dist.pdf(z_grid)), label=r'$p(z)$')
ax_result.plot(z_grid, post_dict['stack'], label=r'$\sum_j p(z | d_{j}$)')
ax_result.plot(z_grid, est_dict['stack'], label=r'$\sum_j p(z | d_{j}, \phi^{*})$')
ax_result.plot(z_grid, true_dict['stack'], label=r'$\sum_j p(z | d_{j}, \phi^{\dagger})$')
#ax_result.errorbar(hist_cents, normed_fit, yerr=normed_errs, label="result")
ax_result.plot(z_grid, model_vals, label='model')
leg = fig_result.legend()


# Fitting in counts space

Here we bin the data space and do the fit by computing the Poisson likelihood for the number of counts in each data bin.  

$\ln \mathcal{L} = \sum_j m_j - \sum_j d_j \log m_j$ 

where

$d_j$ is the number of data counts in bin $j$

$m_j$ is the number of model counts in bin $j$

In [ ]:
N_LIKE_PTS = 301
like_grid = np.linspace(Z_OBS_MIN, Z_OBS_MAX, N_LIKE_PTS)
#eval_bins = np.searchsorted(z_bins, eval_grid, side='left')-1
#eval_mask = (eval_bins >= 0) * (eval_bins < z_bins.size-1)
#eval_grid = eval_grid[eval_mask]
#eval_bins = eval_bins[eval_mask]
#like_eval = likelihood.pdf(like_grid)[eval_bins]
obs_cts_grid = np.linspace(Z_OBS_MIN, Z_OBS_MAX, 7)
data_hist = np.histogram(z_meas_sample, bins=obs_cts_grid)
data_cts = data_hist[0]

In [ ]:
like = qit.BinnedLike(model=model, data_hist=data_hist, estimator=like_estim,\
                      like_grid=like_grid, model_grid=eval_grid)

In [ ]:
flat = 0.5*data_cts.sum()*np.ones(4)
model_flat = like.model_counts(np.log(flat))
model_true = like.model_counts(np.log(true_vals))
ll_flat = like(np.log(flat))
ll_true = like(np.log(true_vals))
print(ll_flat, ll_true)

In [ ]:
result = minimize(like, np.ones(4))
print(result)

In [ ]:
model_cts = like.model_counts(result['x'])
cts_cent = 0.5 * (obs_cts_grid[1:] + obs_cts_grid[:-1])
fig_fit, ax_fit = qp.plotting.make_figure_axes(xlim=(Z_OBS_MIN, Z_OBS_MAX),
                                                     xlabel=r"$d$",
                                                     ylabel=r"$n(d)$")
ax_fit.set_yscale('log')
ax_fit.set_ylim(1., 1e4)
ax_fit.scatter(cts_cent, data_cts, label='data')
ax_fit.plot(cts_cent, model_cts, label='fit')
leg = fig_fit.legend()

In [ ]:
fit_cts = np.exp(result['x'])
fit_cts *= 2/fit_cts.sum()
pdf_true = true_vals * 2 / true_vals.sum()
fig_fit2, ax_fit2 = qp.plotting.make_figure_axes(xlim=(Z_TRUE_MIN, Z_TRUE_MAX),
                                                xlabel=r'$z_{\rm true}$',
                                                ylabel=r'p(z)')

ax_fit2.hist(z_true_sample[mask], bins=hist_bins, density=True, label=r'$z_{\rm true}$', histtype='step')
ax_fit2.plot(z_grid, np.squeeze(true_dist.pdf(z_grid)), label=r'$p(z)$')
ax_fit2.plot(hist_cents, fit_cts, label="fit")
ax_fit2.plot(z_grid, model_vals, label='model')
leg = fig_fit2.legend()
